In [15]:
import itertools
import numpy as np

def matrix(a, b, match_score=3, gap_cost=2):
    H = np.zeros((len(a) + 1, len(b) + 1), int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H

In [16]:
def traceback(H, b, b_='', old_i=0):
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are **last** indexes of H.max()
    if H[i, j] == 0:
        return b_, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_
    return traceback(H[0:i, 0:j], b, b_, i)


In [17]:
def smith_waterman(a, b, match_score=3, gap_cost=2):
    a, b = a.upper(), b.upper()
    H = matrix(a, b, match_score, gap_cost)
    b_, pos = traceback(H, b)
    return pos, pos + len(b_)

In [18]:
print(matrix('ATGCT', 'AGCT'))
a, b = 'atgct', 'agct'
H = matrix(a, b)
print(traceback(H, b))
a, b = 'ATGCT', 'AGCT'
start, end = smith_waterman(a, b)
print(a[start:end])     

[[ 0  0  0  0  0]
 [ 0  3  1  0  0]
 [ 0  1  0  0  3]
 [ 0  0  4  2  1]
 [ 0  0  2  7  5]
 [ 0  0  0  5 10]]
('a-gct', 0)
ATGCT


In [19]:
print(matrix('AATCG', 'AACG'))
a, b = 'aatcg', 'aacg'
H = matrix(a, b)
print(traceback(H, b))
a, b = 'AATCG', 'AACG'
start, end = smith_waterman(a, b)
print(a[start:end])    

[[ 0  0  0  0  0]
 [ 0  3  3  1  0]
 [ 0  3  6  4  2]
 [ 0  1  4  3  1]
 [ 0  0  2  7  5]
 [ 0  0  0  5 10]]
('aa-cg', 0)
AATCG
